In [1]:
import redis
import os
from deepface import DeepFace 
import numpy as np
from tqdm import tqdm
from deepface.commons import functions
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

In [2]:
rep = []
labels = []
for dirpath, dirnames, filenames in os.walk('E:\\tf_openpose\\5celeb\\data\\train\\'):
    for filename in filenames:
        img_path = f"{dirpath}\\{filename}"
        if  ".jpg" or ".png" in img_path:
            embedding_obj = DeepFace.represent(
                img_path=img_path,
                model_name="ArcFace",
                enforce_detection=False
            )
            embedding = embedding_obj[0]["embedding"]
            label = os.path.basename(dirpath)
            rep.append((img_path, embedding))
            labels.append(label)


In [3]:
from redis.commands.search.field import TagField, VectorField, TextField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

from ast import literal_eval
from typing import List, Iterator
import pandas as pd
import time


article_df = pd.DataFrame(rep, columns=['url', 'vector'])
article_df['id'] = np.arange(0, len(article_df))
article_df['labels'] = labels
article_df

,url,vector,id,labels
0,E:\tf_openpose\5celeb\data\train\ben_afflek\ht...,"[-0.3078286647796631, 0.6858863830566406, -0.0...",0,ben_afflek
1,E:\tf_openpose\5celeb\data\train\ben_afflek\ht...,"[-0.030019249767065048, -0.05606715381145477, ...",1,ben_afflek
2,E:\tf_openpose\5celeb\data\train\ben_afflek\ht...,"[0.14424189925193787, 0.01635141484439373, 0.1...",2,ben_afflek
3,E:\tf_openpose\5celeb\data\train\ben_afflek\ht...,"[-0.15198102593421936, 0.1615729033946991, 0.0...",3,ben_afflek
4,E:\tf_openpose\5celeb\data\train\ben_afflek\ht...,"[0.247961163520813, 0.03352166339755058, -0.00...",4,ben_afflek
...,...,...,...,...
109,E:\tf_openpose\5celeb\data\train\mindy_kaling\...,"[-0.031962521374225616, 0.35589727759361267, 0...",109,mindy_kaling
110,E:\tf_openpose\5celeb\data\train\mindy_kaling\...,"[0.06394591182470322, 0.4917104244232178, 0.21...",110,mindy_kaling
111,E:\tf_openpose\5celeb\data\train\mindy_kaling\...,"[-0.06867343187332153, 0.37567126750946045, 0....",111,mindy_kaling
112,E:\tf_openpose\5celeb\data\train\mindy_kaling\...,"[0.18796458840370178, 0.23716022074222565, 0.1...",112,mindy_kaling


In [4]:
REDIS_HOST =  "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = "" # default for passwordless Redis

# Connect to Redis
r = redis.Redis(
    host=REDIS_HOST,
    port=REDIS_PORT,
    password=REDIS_PASSWORD
)

In [18]:
INDEX_NAME = 'embeddings-index'
doc_ = r.ft(INDEX_NAME).search('cdchu2').docs
vector_ = [doc.url for i, doc in enumerate(doc_)]
vector_

['E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\2023-12-06-17-28-11.jpg',
 'E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\2023-12-06-17-28-21.jpg',
 'E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\2023-12-06-17-28-26.jpg',
 'E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\2023-12-06-17-28-31.jpg',
 'E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\2023-12-06-17-28-32.jpg',
 'E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\IMG_2880.png',
 'E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\IMG_2885.png',
 'E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\IMG_2895.png',
 'E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\IMG_3275.png',
 'E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\IMG_3276.png']

In [19]:
from PIL import Image
image = Image.open('E:\\tf_openpose\\5celeb\\data\\train\\cdchu2\\2023-12-06-17-28-11.jpg')


In [16]:
import zlib
import pickle
r.set("key", pd.DataFrame.to_msgpack()(compress='zlib'))
pd.read_msgpack(redisConn.get("key"))


AttributeError: type object 'DataFrame' has no attribute 'to_msgpack'